Jon V

6/2/18

In [50]:
from ase.spacegroup import crystal
from ase.build import *
import numpy as np
import matplotlib.pyplot as plt
import os
import ase
import json

### Choose inputs.

In [51]:
# define md parameters
pref = 'si'
pseudo_dir = '/n/home03/jonpvandermause/qe-6.2.1/pseudo'
outdir='/n/home03/jonpvandermause/Cluster/Si_Supercell_MD'
alat = 5.431 # lattice parameter of si in angstrom
ecut = 18.0 # plane wave cutoff energy
nk = 4 # size of kpoint grid
dim = 3 # size of supercell
nat = 2 * dim**3 # number of atoms in supercell
dt = 20 # MD time step in Rydberg atomic units
nstep = 1000 # number of MD steps
pert_size = 0.01 * alat # size of initial perturbation

# specify locations of QE codes
npool = 36 # number of k-pt pools
mult = 10
ncpu = npool * mult # number of cores
nodes = int(np.ceil(ncpu/32)) # number of nodes
pw_loc = '/n/home03/jonpvandermause/qe-6.2.1/bin/pw.x'
in_name = 'si.md.in'
out_name = 'si.md.out'
sh_name = 'Si_MD.sh'
partition = 'kozinsky'
memory = 1000
email = 'jonathan_vandermause@g.harvard.edu'

### Make ASE structure class. (From AP275 labs.)

In [52]:
class Generic(object):
    def __init__(self, *args, **kwargs):
        if args:
            # in case of a single literal or a dict being supplied
            assert not kwargs and len(args) == 1, "wrong number of init args"
            self.content = args[0]
        if kwargs:
            # can supply content=dict, or an unpacked dict by keywords, but not both
            if 'content' in kwargs:
                assert not args and len(kwargs) == 1, "wrong number of init kwargs"
                self.content = kwargs['content']
            else:
                self.content = kwargs

    def __repr__(self):
        return "{} {}".format(self.__class__.__name__, self.content)

    
class Param(Generic):
    """Class representing a dictionary of parameters"""
    pass


class Struc(Param):
    """
    Data class containing information about a structure
    example:
        struc1 = {"cell": [[1.0, 0, 0],[0, 1.0, 0],[0, 0, 2.0]],
                  "periodicity" : [True, True, True],
                  "species": {'H': {'id': 1, 'mass':1.008, 'atomic_number': 1},
                              'He',{'id': 2, 'mass': 4.003, 'atomic_number' : 2}}
                  "positions": [['H', [4.0, 3.0, 6.0]],
                                ['He', [4.0, 5.0, 9.0]]],
                 }
    """
    @staticmethod
    def from_ase(aseobj):
        # need to use method tolist() of numpy arrays to get valid json
        cell = aseobj.cell.tolist()
        pbc = aseobj.get_pbc().tolist()
        symbols = aseobj.get_chemical_symbols()
        masses = aseobj.get_masses()
        positions = aseobj.get_positions().tolist()
        # easy way to get rid of tuples after zipping
        positions = json.loads(json.dumps(list(zip(symbols, positions))))
        types = sorted(list(set(zip(symbols, masses))))
        species = {tp[0]: {'mass': tp[1], 'kind': i + 1} for i, tp in enumerate(types)}
        content = {'cell': cell, 'positions': positions, 'periodicity': pbc, 'species': species}
        return content

    def to_ase(self):
        cell = self.content['cell']
        atoms = [ase.Atom(site[0], tuple(site[1])) for site in self.content['positions']]
        aseobj = ase.Atoms(atoms)
        aseobj.set_cell(cell)
        aseobj.set_pbc(self.content['periodicity'])
        return aseobj

    @property
    def symbols(self):
        return [s[0] for s in self.content['positions']]

    @property
    def cell(self):
        return self.content['cell']

    @property
    def positions(self):
        return self.content['positions']

    @property
    def n_atoms(self):
        return len(self.content['positions'])

    @property
    def n_species(self):
        return len(self.species)

    @property
    def species(self):
        return self.content['species']
    
def ase2struc(ase_atoms):
    return Struc.from_ase(ase_atoms)


def struc2ase(struc):
    return Struc.to_ase(struc)

def write_file(fname, text):
    with open(fname, 'w') as fin:
        fin.write(text)

### Make Si supercell.

In [53]:
# create unperturbed silicon supercell
def make_struc_super(alat, dim):
    unitcell = crystal('Si', [(0, 0, 0)], spacegroup=227, \
                       cellpar=[alat, alat, alat, 90, 90, 90], \
                      primitive_cell = True)
    multiplier = np.identity(3) * dim
    ase_supercell = make_supercell(unitcell, multiplier)
    structure = Struc(ase2struc(ase_supercell))
    return structure

# perturb the positions of a supercell
def perturb_struc(alat, dim, pert_size):   
    struc_pert = make_struc_super(alat, dim)
    for n in range(len(struc_pert.content['positions'])):
        for m in range(3):
            # get current coordinate
            coord_curr = struc_pert.content['positions'][n][1][m]

            # get perturbation by drawing from uniform
            pert = np.random.uniform(-pert_size, pert_size)

            # perturb the coordinate
            struc_pert.content['positions'][n][1][m] += pert
            
    return struc_pert

# get supercell positions
def get_position_txt(struc):
    
    # write atomic positions
    postxt = ''
    postxt += 'ATOMIC_POSITIONS {angstrom}'
    for index, positions in enumerate(struc.content['positions']):
        postxt += '\n {} {:1.5f} {:1.5f} {:1.5f}'.format(positions[0], *positions[1])
        
    # write cell parameters
    celltxt = ''
    celltxt += 'CELL_PARAMETERS {angstrom}'
    for vector in struc.content['cell']:
        celltxt += '\n {:1.5f} {:1.5f} {:1.5f}'.format(*vector)
    return postxt, celltxt

### Make MD file.

In [54]:
# function that creates scf input
def get_md_text(pref, pseudo_dir, outdir, alat,\
                ecut, nk, dim, nat, dt, nstep, pert_size):
    
    # get perturbed structure
    struc = perturb_struc(alat, dim, pert_size)
    pos, cell = get_position_txt(struc)
    
    md_text = """ &control
    calculation = 'md'
    restart_mode = 'from_scratch'
    pseudo_dir = '{1}'
    outdir = '{2}'
    dt = {8}
    nstep = {9}
    disk_io = 'high'
 /
 &system
    ibrav= 0
    nat= {3}
    ntyp= 1
    ecutwfc ={4}
    nosym = .true.
 /
 &electrons
    conv_thr =  1.0d-10
    mixing_beta = 0.7
 /
 &ions
    pot_extrapolation = 'second-order'
    wfc_extrapolation = 'second-order'
 /
ATOMIC_SPECIES
 Si  28.086  Si.pz-vbc.UPF
{5}
{6}
K_POINTS automatic
 {7} {7} {7}  0 0 0
    """.format(pref, pseudo_dir, outdir, \
               nat, ecut, cell, pos, nk, dt, nstep)
    
    return md_text

In [55]:
# make text
md_text = get_md_text(pref, pseudo_dir, outdir, alat,\
                ecut, nk, dim, nat, dt, nstep, pert_size)
write_file(in_name, md_text)

### Make SH file.

In [56]:
def make_sh_text(ncpu, nodes, pw_loc, in_file, out_file, npool, partition, memory, email):

    sh_text = """#!/bin/sh
#SBATCH -n {0}
#SBATCH -N {1}
#SBATCH -t 1-00:00
#SBATCH -e test.err
#SBATCH -p {6}
#SBATCH -o test.out
#SBATCH --mem-per-cpu={7}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={8}

module load gcc/4.9.3-fasrc01 openmpi/2.1.0-fasrc01
module load python/3.6.3-fasrc01

MPI="mpirun"
PW="{2}"

$MPI $PW -npool {5} < {3} > {4}
""".format(ncpu, nodes, pw_loc, in_file, out_file, npool, partition, memory, email)
    
    return sh_text

In [57]:
# make sh text
sh_text = make_sh_text(ncpu, nodes, pw_loc, in_name, out_name, npool, partition, memory, email)
write_file(sh_name, sh_text)

### Parse the MD output file.

In [ ]:
def parse_qe_pwscf_output(outfile):
    with open(outfile.path, 'r') as outf:
        for line in outf:
            if line.lower().startswith('     pwscf'):
                walltime = line.split()[-3] + line.split()[-2]
            if line.lower().startswith('     total force'):
                total_force = float(line.split()[3]) * (13.605698066 / 0.529177249)
            if line.lower().startswith('!    total energy'):
                total_energy = float(line.split()[-2]) * 13.605698066
            if line.lower().startswith('          total   stress'):
                pressure = float(line.split()[-1])
    result = {'energy': total_energy, 'force': total_force, 'pressure': pressure}
    return result